# BeemaBot Model Training

In [2]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json
import csv

### Importing our dataset

In [3]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

### Bulding our list of words, classes and words to tags mapping(documents)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize and lower each word
lemma_function = WordNetLemmatizer()
words = [lemma_function.lemmatize(w.lower()) for w in words if w not in ignore_words]

# remove duplicates
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))

# words = all words, vocabulary
print (len(words), "lemmatized words")
# documents = combination between patterns and intents
print (len(documents), "word to tag mappings")
# classes = intents
print (len(classes), "classes")

2934 lemmatized words
11130 word to tag mappings
175 classes


### Creating training data from dataset

In [5]:
training = []
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    # lemmatize each word
    pattern_words = [lemma_function.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

/var/folders/fj/l84qvx552y37ht23spc9y0140000gn/T/ipykernel_87264/1985779988.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


### Building the Neural Network Model

In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
# Adding Dropout to avoid overfitting
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

2021-11-21 01:26:01.454585: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

### Training the model on training data

In [8]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
2226/2226 [==============================] - 6s 2ms/step - loss: 3.9420 - accuracy: 0.1351
Epoch 2/200
2226/2226 [==============================] - 5s 2ms/step - loss: 2.5670 - accuracy: 0.3594
Epoch 3/200
2226/2226 [==============================] - 5s 2ms/step - loss: 2.1698 - accuracy: 0.4356
Epoch 4/200
2226/2226 [==============================] - 5s 2ms/step - loss: 2.0285 - accuracy: 0.4792
Epoch 5/200
2226/2226 [==============================] - 5s 2ms/step - loss: 1.9191 - accuracy: 0.5049
Epoch 6/200
2226/2226 [==============================] - 5s 2ms/step - loss: 1.8627 - accuracy: 0.5159
Epoch 7/200
2226/2226 [==============================] - 5s 2ms/step - loss: 1.7881 - accuracy: 0.5423
Epoch 8/200
2226/2226 [==============================] - 6s 2ms/step - loss: 1.7793 - accuracy: 0.5406
Epoch 9/200
2226/2226 [==============================] - 5s 2ms/step - loss: 1.7711 - accuracy: 0.5437
Epoch 10/200
2226/2226 [==============================] - 5s 2ms/step - l

### Functions to preprocess input

In [9]:
def get_words_from_input_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemma_function.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def get_input_vector(sentence, words, show_details=True):
    sentence_words = get_words_from_input_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

### Taking input message and converting to input vector

In [10]:
input_sentence = "get car insurance"

In [11]:
input_vector = get_input_vector(input_sentence, words)
print ("Sentence as input vector - ", input_vector)

found in bag: get
found in bag: car
found in bag: insurance
Sentence as input vector -  [0 0 0 ... 0 0 0]


Predict output class using our model

In [12]:
inputvar = pd.DataFrame([input_vector], dtype=float, index=['input'])
model_output = model.predict(inputvar)

Get tag with maximum probability

In [13]:
print("Input sentence - ", input_sentence)
print("Output class for input - ", classes[model_output.argmax()])

Input sentence -  get car insurance
Output class for input -  INFO_GET_A_QUOTE_AUTO


Save model and data structures to pickle files

In [14]:
pickle.dump(model, open("chatbot-model.pkl", "wb"))

2021-11-21 01:42:10.926097: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://3b7b6017-b113-466d-ac9a-16bedc61c371/assets


In [15]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "chatbot-data.pkl", "wb" ) )

# Testing the model

### Loading in model and data structures

In [16]:
data = pickle.load( open( "chatbot-data.pkl", "rb" ) )
words = data['words']
classes = data['classes']

# Load intents data
intents =  json.load(open('intents.json'))['intents']
lemma_function = WordNetLemmatizer()

#load model
with open(f'chatbot-model.pkl', 'rb') as f:
    model = pickle.load(f)

### Function to classify input

In [17]:
def classify_input(sentence):
    input_data = pd.DataFrame([get_input_vector(sentence, words, False)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    results = [[i,r] for i,r in enumerate(results)]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        response = None
        context = None
        for intent in intents:
            if intent['tag'] == classes[r[0]]:
                response = intent['responses']
                context = intent['context']

        return_list.append({'tag': classes[r[0]], 'probability': str(r[1]), 'response': response, 'context': context})
    
    return return_list

### Loading in testing data and predicting intent for all testing samples

In [18]:
csvFilePath = r'testing_data.csv'

total = 0
correct = 0

with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        for rows in csvReader:
            sentence = rows['UTTERANCES']
            response = classify_input(sentence)
            tag = response[0]['tag'] if response[0] else ''
            if tag == rows['INTENT_NAME']:
                correct+=1
            total+=1
            
            if total%500 == 0:
                print(f'{total} samples tested, accuracy: {(correct/total) *100} %')

print(f'{total} samples tested, final accuracy of model: {correct/total * 100}')

500 samples tested, accuracy: 85.0 %
1000 samples tested, accuracy: 75.9 %
1500 samples tested, accuracy: 79.80000000000001 %
2000 samples tested, accuracy: 82.1 %
2500 samples tested, accuracy: 82.52000000000001 %
3000 samples tested, accuracy: 82.93333333333334 %
3500 samples tested, accuracy: 81.65714285714286 %
4000 samples tested, accuracy: 82.125 %
4500 samples tested, accuracy: 82.77777777777777 %
5000 samples tested, accuracy: 82.67999999999999 %
5500 samples tested, accuracy: 82.7090909090909 %
6000 samples tested, accuracy: 83.39999999999999 %
6500 samples tested, accuracy: 83.03076923076922 %
7000 samples tested, accuracy: 83.68571428571428 %
7500 samples tested, accuracy: 84.05333333333334 %
8000 samples tested, accuracy: 84.3875 %
8500 samples tested, accuracy: 84.14117647058823 %
9000 samples tested, accuracy: 84.23333333333333 %
9500 samples tested, accuracy: 83.98947368421052 %
10000 samples tested, accuracy: 83.87 %
10500 samples tested, accuracy: 84.04761904761905 %
1

## Final accuracy of model: 83.5 %